导入数据

In [34]:
import os
import pandas as pd
DATASET_DIR = './'
df = pd.read_excel( '作文打分语料（英语）.xlsx', sheet_name= 0)
y = df['分数']
X = df

print(type(X))
print(y)

<class 'pandas.core.frame.DataFrame'>
0        8
1       10
2        8
3       11
4        9
        ..
1195     9
1196     8
1197     8
1198     8
1199    11
Name: 分数, Length: 1200, dtype: int64


In [17]:
X.head()

,编号,作文,分数
0,1,"Dear Local Newspaper, I believe the computer d...",8
1,2,"Dear @CAPS1 @CAPS2, I have heard the concern o...",10
2,3,@CAPS1 you know we all like computers and we a...,8
3,4,I Believe that computers have a positive effec...,11
4,5,"Dear newstimes, I really think that computers ...",9


数据预处理

In [9]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [10]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\FengYJ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

定义模型 两层LSTM

In [11]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

Using TensorFlow backend.


训练模型

In [32]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import mean_squared_error
from math import sqrt

cv = KFold(n_splits = 5, shuffle = True)
results = []
y_pred_list = []

count = 1
for traincv, testcv in cv.split(X):
    print("\n--------Fold {}--------\n".format(count))
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
#     print(X_train)
    train_essays = X_train['作文']
    test_essays = X_test['作文']
    
    sentences = []
    
    for essay in train_essays:
            # Obtaining all sentences from the training essays.
            sentences += essay_to_sentences(essay, remove_stopwords = True)
    
    print(sentences[0])
    
    # Initializing variables for word2vec model.
    num_features = 300 
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

    # Generate training and testing data word vectors.
    clean_train_essays = []
    for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
    #print(train_essays)
    #print(clean_train_essays[0])
    #print(len(clean_train_essays))
    
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    #print(trainDataVecs[0])
    #print(len(trainDataVecs))
    #print(trainDataVecs.shape)
    
    
    clean_test_essays = []
    for essay_v in test_essays:
        clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
    testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=1000)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    y_pred = lstm_model.predict(testDataVecs)
    
    # Save any one of the 5 models.
    if count == 5:
         lstm_model.save('./model_weights/final_lstm.h5')
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    kappa = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    df2 = pd.DataFrame()
    df2['true'] = y_test.values
    df2['pred'] = y_pred
    spear = df2.corr('spearman')['pred'][0]
    pearson = df2.corr('pearson')['pred'][0]
    RMSE = sqrt(mean_squared_error(y_test.values, y_pred))
    print()
    print("Spearman’s ρ Score:", spear)
    print("Pearson r Score:", pearson)
    print("RMSE Score:", RMSE)
    print("Cohen’s κ Score:", kappa)
   # print(y_test.values)
    #print(y_pred)
    
    results.append(kappa)
    
    count += 1
    break


--------Fold 1--------

['dear', 'local', 'newspaper', 'believe', 'computer', 'positive', 'perks', 'also', 'negative', 'perks']
Training Word2Vec Model...


D:\other-software\Anaconda\envs\nlp37\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_27 (LSTM)               (None, 1, 300)            721200    
_________________________________________________________________
lstm_28 (LSTM)               (None, 64)                93440     
_________________________________________________________________
dropout_14 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
960/960 [==============================] - 1s 765us/step - loss: 52.8578 - mae: 6.9574
Epoch 2/1000
960/960 [==============================] - 0s 212us/step - loss: 8.8990 - mae: 2.4764
Epoch 3/1000
960/960 [========================

960/960 [==============================] - 0s 204us/step - loss: 3.4302 - mae: 1.4252
Epoch 75/1000
960/960 [==============================] - 0s 198us/step - loss: 3.1878 - mae: 1.3880
Epoch 76/1000
960/960 [==============================] - 0s 199us/step - loss: 3.1196 - mae: 1.3793
Epoch 77/1000
960/960 [==============================] - 0s 198us/step - loss: 3.1448 - mae: 1.3977
Epoch 78/1000
960/960 [==============================] - 0s 193us/step - loss: 3.2129 - mae: 1.3851
Epoch 79/1000
960/960 [==============================] - 0s 195us/step - loss: 3.3142 - mae: 1.4091
Epoch 80/1000
960/960 [==============================] - 0s 198us/step - loss: 3.2064 - mae: 1.3879
Epoch 81/1000
960/960 [==============================] - 0s 198us/step - loss: 3.2140 - mae: 1.4046
Epoch 82/1000
960/960 [==============================] - 0s 197us/step - loss: 3.3194 - mae: 1.4067
Epoch 83/1000
960/960 [==============================] - 0s 196us/step - loss: 2.7156 - mae: 1.2759
Epoch 84/1000


960/960 [==============================] - 0s 204us/step - loss: 2.9882 - mae: 1.3653
Epoch 156/1000
960/960 [==============================] - 0s 198us/step - loss: 2.8731 - mae: 1.3339
Epoch 157/1000
960/960 [==============================] - 0s 197us/step - loss: 2.7793 - mae: 1.3126
Epoch 158/1000
960/960 [==============================] - 0s 201us/step - loss: 2.9935 - mae: 1.3703
Epoch 159/1000
960/960 [==============================] - 0s 197us/step - loss: 2.9230 - mae: 1.3331
Epoch 160/1000
960/960 [==============================] - 0s 197us/step - loss: 2.6319 - mae: 1.2758
Epoch 161/1000
960/960 [==============================] - 0s 199us/step - loss: 2.8026 - mae: 1.3231
Epoch 162/1000
960/960 [==============================] - 0s 194us/step - loss: 2.8964 - mae: 1.3225
Epoch 163/1000
960/960 [==============================] - 0s 196us/step - loss: 2.7685 - mae: 1.3159
Epoch 164/1000
960/960 [==============================] - 0s 287us/step - loss: 2.9045 - mae: 1.3346
Epoch

...

960/960 [==============================] - 0s 211us/step - loss: 2.2829 - mae: 1.1885
Epoch 642/1000
960/960 [==============================] - 0s 209us/step - loss: 2.2391 - mae: 1.1803
Epoch 643/1000
960/960 [==============================] - 0s 207us/step - loss: 2.3003 - mae: 1.1933
Epoch 644/1000
960/960 [==============================] - 0s 209us/step - loss: 2.2878 - mae: 1.1906
Epoch 645/1000
960/960 [==============================] - 0s 208us/step - loss: 2.1993 - mae: 1.1460
Epoch 646/1000
960/960 [==============================] - 0s 211us/step - loss: 2.2560 - mae: 1.1875
Epoch 647/1000
960/960 [==============================] - 0s 207us/step - loss: 2.1995 - mae: 1.1470
Epoch 648/1000
960/960 [==============================] - 0s 208us/step - loss: 2.2295 - mae: 1.1712
Epoch 649/1000
960/960 [==============================] - 0s 207us/step - loss: 2.3523 - mae: 1.1819
Epoch 650/1000
960/960 [==============================] - 0s 204us/step - loss: 2.3662 - mae: 1.2015
Epoch

960/960 [==============================] - 0s 212us/step - loss: 2.2171 - mae: 1.1495
Epoch 723/1000
960/960 [==============================] - 0s 206us/step - loss: 2.2153 - mae: 1.1480
Epoch 724/1000
960/960 [==============================] - 0s 208us/step - loss: 2.1632 - mae: 1.1389
Epoch 725/1000
960/960 [==============================] - 0s 209us/step - loss: 2.1221 - mae: 1.1410
Epoch 726/1000
960/960 [==============================] - 0s 210us/step - loss: 2.0513 - mae: 1.1247
Epoch 727/1000
960/960 [==============================] - 0s 207us/step - loss: 2.1528 - mae: 1.1491
Epoch 728/1000
960/960 [==============================] - 0s 208us/step - loss: 2.1285 - mae: 1.1278
Epoch 729/1000
960/960 [==============================] - 0s 204us/step - loss: 2.2371 - mae: 1.1746
Epoch 730/1000
960/960 [==============================] - 0s 209us/step - loss: 2.2636 - mae: 1.1763
Epoch 731/1000
960/960 [==============================] - 0s 209us/step - loss: 2.1104 - mae: 1.1236
Epoch

960/960 [==============================] - 0s 210us/step - loss: 2.0810 - mae: 1.1080
Epoch 804/1000
960/960 [==============================] - 0s 207us/step - loss: 2.1612 - mae: 1.1498
Epoch 805/1000
960/960 [==============================] - 0s 212us/step - loss: 2.1839 - mae: 1.1437
Epoch 806/1000
960/960 [==============================] - 0s 208us/step - loss: 2.0702 - mae: 1.1217
Epoch 807/1000
960/960 [==============================] - 0s 205us/step - loss: 2.1118 - mae: 1.1308
Epoch 808/1000
960/960 [==============================] - 0s 209us/step - loss: 2.1941 - mae: 1.1559
Epoch 809/1000
960/960 [==============================] - 0s 212us/step - loss: 2.0834 - mae: 1.1308
Epoch 810/1000
960/960 [==============================] - 0s 207us/step - loss: 2.0676 - mae: 1.1257
Epoch 811/1000
960/960 [==============================] - 0s 207us/step - loss: 2.0831 - mae: 1.1160
Epoch 812/1000
960/960 [==============================] - 0s 196us/step - loss: 2.0018 - mae: 1.1082
Epoch

960/960 [==============================] - 0s 208us/step - loss: 2.1183 - mae: 1.1278
Epoch 884/1000
960/960 [==============================] - 0s 207us/step - loss: 2.0347 - mae: 1.0993
Epoch 885/1000
960/960 [==============================] - 0s 210us/step - loss: 2.0804 - mae: 1.1320
Epoch 886/1000
960/960 [==============================] - 0s 213us/step - loss: 2.0769 - mae: 1.1135
Epoch 887/1000
960/960 [==============================] - 0s 204us/step - loss: 1.9742 - mae: 1.0905
Epoch 888/1000
960/960 [==============================] - 0s 206us/step - loss: 2.0325 - mae: 1.1133
Epoch 889/1000
960/960 [==============================] - 0s 207us/step - loss: 2.0757 - mae: 1.1155
Epoch 890/1000
960/960 [==============================] - 0s 208us/step - loss: 2.0878 - mae: 1.1270
Epoch 891/1000
960/960 [==============================] - 0s 208us/step - loss: 2.0868 - mae: 1.1278
Epoch 892/1000
960/960 [==============================] - 0s 204us/step - loss: 2.1336 - mae: 1.1236
Epoch

Epoch 964/1000
960/960 [==============================] - 0s 207us/step - loss: 1.9937 - mae: 1.1002
Epoch 965/1000
960/960 [==============================] - 0s 205us/step - loss: 1.9412 - mae: 1.0757
Epoch 966/1000
960/960 [==============================] - 0s 210us/step - loss: 2.0429 - mae: 1.1240
Epoch 967/1000
960/960 [==============================] - 0s 207us/step - loss: 1.9245 - mae: 1.0739
Epoch 968/1000
960/960 [==============================] - 0s 205us/step - loss: 1.9606 - mae: 1.0844
Epoch 969/1000
960/960 [==============================] - 0s 208us/step - loss: 1.9788 - mae: 1.0990
Epoch 970/1000
960/960 [==============================] - 0s 205us/step - loss: 2.0953 - mae: 1.1254
Epoch 971/1000
960/960 [==============================] - 0s 208us/step - loss: 1.9683 - mae: 1.0774
Epoch 972/1000
960/960 [==============================] - 0s 208us/step - loss: 2.0278 - mae: 1.1107
Epoch 973/1000
960/960 [==============================] - 0s 198us/step - loss: 2.0002 - ma